In [15]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sb

In [ ]:
df = pd.read_excel("./dataset.xlsx")

In [ ]:
np.random.seed(21)

pd.set_option("display.max_columns", None)      # zobrazenie vsetkych stlpcov
pd.set_option("display.width", 200)             # rozsirenie sirky vypisu
pd.set_option("display.max_rows", 100)          # viacej riadkov

In [17]:
df.columns = (
    df.columns
    .str.replace(r"[.: ]", "_", regex=True)
    .str.replace(r"_+", "_", regex=True)
    .str.strip("_")
)

In [18]:
print(df.columns)

Index(['Unnamed_0_1', 'Unnamed_0', 'PateID', 'Gender', 'DOB', 'CXL_YES', 'CXL_date', 'DatasetID', 'Atopy', 'Eczema', 'Hayfever', 'Asthma', 'Allergies', 'Rubbing', 'EXAM', 'Exam_date', 'kMax',
       'PachyMin', 'ak1', 'ak2', 'KmF', 'Axis_F_(flat)', 'Astig_F_(D)', 'R_Per_F_(mm)', 'Rmin', 'Num_Ecc_F', 'Asph_Q_F', 'Rf_B_(mm)', 'Rs_B_(mm)', 'Rh_B_(mm)', 'Rv_B_(mm)', 'pk1', 'pk2', 'Rm_B_(mm)',
       'Km_B_(mm)', 'Axis_B_(flat)', 'Astig_B_(D)', 'R_Per_B_(mm)', 'Rmin_B', 'Num_Ecc_B', 'Asph_Q_B', 'Ecc_Zone', 'Pachy_Apex', 'Pachy_Pupil', 'Pupil_Pos_X', 'Pupil_Pos_Y', 'Pachy_Min_Pos_X',
       'Pachy_Min_Pos_Y', 'C_Vol_D_3mm', 'C_Vol_D_5mm', 'C_Vol_D_7mm', 'C_Vol_D_10mm', 'ISV', 'IVA', 'KI', 'CKI', 'IHA', 'IHD', 'RSagMin', 'D0mm_Pachy', 'D2mm_Pachy', 'D4mm_Pachy', 'D6mm_Pachy',
       'D8mm_Pachy', 'D10mm_Pachy', 'D0mm_Prog', 'D2mm_Prog', 'D4mm_Prog', 'D6mm_Prog', 'D8mm_Prog', 'D10mm_Prog', 'LC_id', 'age_at_exam', 'Before_After_CXL', 'atopy_dummy', 'Rubbing_dummy',
       'atopy_rubbing', 'Days

## Spracovanie atributov

In [19]:
attrib_want_remove: list[str] = ["Unnamed_0_1", "Unnamed_0", "Exam_name", "Pupil_Pos_X",
                                 "Pupil_Pos_Y", "CXL_date", "CXL_YES", "DatasetID",
                                 "Before_After_CXL"]

df = df.drop(attrib_want_remove,
             axis=1
             )

#### Preco prave tieto stlpce?
- `Unnamed_0_1`: nemame o nom ziadne informacie v povodnej dokumentacii (nevieme co data znamenaju)
- `Unnamed_0`: nemame o nom ziadne informacie v povodnej dokumentacii (nevieme co data znamenaju)
- `Exam_name`: nemame o nom ziadne informacie v povodnej dokumentacii (nevieme co data znamenaju)
- `Pupil_Pos_X`: velmi vela zaznamov nema tieto hodnoty (zistit %)
- `Pupil_Pos_Y`: velmi vela zaznamov nema tieto hodnoty (zistit %)
- `CXL_date`: nemyslim si, ze tento datum je potrebny v mojej situacii, lebo mi nedava (aspon v ton nevidim) dolezitu informaciu
- `CXL_YES`: nemyslim si, ze mat informaciu ci pacient mal alebo nikdy nemal spraveny CLX
- `DatasetID`: neviem si predstavit, ze ako by mi informacie ohladom data tychto dat pomohla urcit vystup
- `Before_After_CXL`: neviem ako by mi informacia ohladom toho, ci bolo meranie vykonane pred alebo po CXL pomohla predikovat hodnotu

In [20]:
# V zadani som mal informaciu, ze ak je hodnota NaN, tak by mala byt 0
df["Rubbing"] = df["Rubbing"].fillna(0)

# PachyMin ma len jednu hodnotu NaN, ja by som to osobne (zaznan) odstranil...

In [21]:
nan_percent = df.isna().mean() * 100
columns_over_20: list[str] = nan_percent[nan_percent > 20]  # type: ignore
columns_over_20 = columns_over_20.index.tolist()            # type: ignore
columns_over_20.remove("referred_CXL")

print(columns_over_20)

df = df.drop(columns_over_20,
             axis=1
             )

['kMaxDiff_N', 'Crit_kMax_N', 'PachyMinDiff', 'PachyMinPercentChange_Exams', 'PachyMinPercentChange_Start', 'Crit_PachyMin_N', 'Crit_Both_N']


In [42]:
df.isna().sum()[df.isna().sum() > 0]

PachyMin           1
referred_CXL    2518
dtype: int64

In [23]:
gender_check = (
    df
    .groupby("PateID")["Gender"]
    .agg(
        total_records="size",
        nan_gender=lambda x: x.isna().sum()
    )
    .query("nan_gender > 0")
)

print(gender_check)

        total_records  nan_gender
PateID                           
65                  2           2
205                 2           2
320                 2           2
321                 3           3
757                 3           3
856                 9           9
875                 5           5
904                 2           2
911                 5           5
992                 5           5


- 38 hodnot bolo v stlpci "Gender". Chcel som overit, ze ci nahodou nedoslo ku chybe pri vkladani dat (jeden pacient mohol mat 3 merania a v jednom z nich mohlo byt Gender=Nan, a v inom Gender=1). Po overeni som zistil, ze vsetci pacienti pre ktorych sa naslo aspon jeden zaznam Gender=Nan maju vsetky zaznamy s Nan. Preto si mozem dovolit odstranit tieto zaznamy

In [24]:
bad_ids = gender_check.index
df = df[~df["PateID"].isin(bad_ids)]

In [25]:
d10_prog_check = (
    df
    .groupby("PateID")["D10mm_Prog"]
    .agg(
        total_records="size",
        d10_prog=lambda x: x.isna().sum()
    )
    .query("d10_prog > 0")
)

print(d10_prog_check)

d8_prog_check = (
    df
    .groupby("PateID")["D8mm_Prog"]
    .agg(
        total_records="size",
        d8_prog=lambda x: x.isna().sum()
    )
    .query("d8_prog > 0")
)

print(d8_prog_check)

        total_records  d10_prog
PateID                         
152                 8         1
177                 6         1
821                 7         1
826                 7         1
916                10         1
1017               11         2
1026               12         2
        total_records  d8_prog
PateID                        
821                 7        1
826                 7        1
916                10        1
1017               11        2
1026               12        2


- Kedze chyba len jeden zaznam pri niekotrych atributov, nebude problem dopocitat ho pomocou priemeru. Taktiez, tyto pacienti maju viac zaznamov, cize to nebude velky problem

In [26]:
cols = ["D10mm_Prog", "D8mm_Prog"]

for col in cols:
    df[col] = (
        df
        .groupby("PateID")[col]
        .transform(lambda x: x.fillna(x.mode().iloc[0]))
    )


#### Odstranenie stlpcov, v kotrych chyba viac ako 20%
- Malo dat, nevieme dopocitat = zbytocne stlpce, nedaju sa pouzit

In [43]:
random_patient = (
    df["PateID"]
    .dropna()
    .unique()
)

random_patient = np.random.choice(
    random_patient,
    size=10,
    replace=False
)

df_sample = df[df["PateID"].isin(random_patient)]

df_sample = df_sample.sort_values(by=["PateID"])
df_sample.to_excel("./test.xlsx")


## Praca s atributmi
- [x] "Gender": ak ma pacient viac zaznamov ale v jednom je NaN, tak mozem sa pokusit zistit, aky Gender mal na ostatnych
- [] "DOB + age_at_exam": skontrolovat atribut s vekom, pripadne ho upravit
- [] "reffered_CXL": ak ma pacient viacej merani, tak pre NaN hodnoty bude vysledok poslendej (Co ak ma vsetky NaN? Potom odstranujeme zaznam)
- [] skontrolovat stlpce s datum, ci su potrebne (nechcem ich tam, ale treba to pozriet)
- [] skontrolovat, ci su nejake stlpce s id, ktore nepotrebujem
- [] pozriet sa na stlpce s casom, nechem ich tam